In [ ]:
import pandas as pd
import json
import numpy as np
import plotly.express as px
# Instalar plotly con la siguiente ruta: pip install plotly

In [ ]:

#%% Opciones para visualizar data frames en consola

pd.set_option("display.max_rows",5000)
pd.set_option("display.max_columns",500)
pd.set_option("display.width",1000)

# Ingreso de datos de entrada: leer archivo JSON

archivo = "archivo_tradeview_0.json"

def f_datosent(P0_archivo="archivo_tradeview_0.json"):
    
    """
    param P0_archivio: Nombre del archivo con ubicación en la misma carpeta.
    """
    
    P1_tipo = P0_archivo.split(".")
    P1_tipo = P1_tipo[1]

    if P1_tipo=="json":
        with open(archivo, encoding="utf-8-sig") as json_file:
            datos_2 = json.load(json_file)["data"]
            
        df_datos = pd.DataFrame(datos_2["closedTransactions"]["list"])
        # Resetear index para tener todos los números de índices completos
        df_datos = df_datos.drop("info",axis=1).dropna().reset_index().drop("index",axis=1)
        
        # Tomar deseados
        df_datos = df_datos[(df_datos["type"]=="buy") | (df_datos["type"]=="sell") | (df_datos["type"]=="s/l") | (df_datos["type"]=="t/p")]
        
        # Renombrar
        
        df_datos.rename(columns  = {"SL":"sl","TP":"tp","price":"openPrice","price2":"closePrice","item":"instrument"},inplace=True) # Con inplace se guarda 
        
        return df_datos

    elif P1_tipo=="xlsx":
        
    
        df_datos = pd.read_excel(archivo)
        
        return df_datos
    
    elif P1_tipo=="csv":
        
        df_datos = pd.read_excel(archivo)
        
        return df_datos
    else:
        
        print("No manejamos tu tipo de extensión")
         
        
    
    
df_datos = f_datosent(P0_archivo=archivo)

In [ ]:
#convertir formatos de columnas a flotantes
df_datos[['profit','size']] = df_datos[['profit','size']].astype(float) 

lista1 =[]
lista2 =[]
h=list(df_datos.loc[:,'instrument'].unique())
h2=df_datos.loc[:,'instrument'].unique()
#creando data frame
x=[]
for i in range(len(h)):
    a=df_datos.where(df_datos.instrument == h[i]).dropna()[['instrument','size','profit']]
    #rezgar profit y size
    a['sizet1']=a[['size']].shift(1)
    a['profit']=a[['profit']].shift(1)
    
    

    condiciones = [a.profit<0,a.profit>0]
    elecciones =['perdió','ganó']
    b=np.select(condiciones,elecciones)
    a['gano_perdio']=b
    #a.head()
    
    c=[]
    for i in range(len(a[['size']])):
        if a.iloc[i,1] > a.iloc[i,3]:
            d = 'aumentó'
        elif a.iloc[i,1] < a.iloc[i,3]:
            d = 'disminuyó'
        elif a.iloc[i,1] == a.iloc[i,3]:
            d = 'mantuvo'
        else:
            d = 'na'
        c.append(d)

    a['com_size']=c
    
    
    
    condiciones = [(a.gano_perdio=='ganó') & (a.com_size=='mantuvo'),(a.gano_perdio=='ganó') & (a.com_size=='aumentó'),
                  (a.gano_perdio=='ganó') & (a.com_size=='disminuyó'),(a.gano_perdio=='perdió') & (a.com_size=='aumentó'),
                  (a.gano_perdio=='perdió') & (a.com_size=='mantuvo'),(a.gano_perdio=='perdió') & (a.com_size=='disminuyó')]
    elecciones =['Razonable','Razonable','Sesgado','Sesgado','Sesgado','Razonable']

    b=np.select(condiciones,elecciones)

    a['Comportamiento']=b

    a=a.iloc[1:]    
    
    tabla1 = a.pivot_table(index='gano_perdio', columns=['instrument','com_size'], aggfunc={'com_size':len})
    tabla2 = a.pivot_table(columns=['instrument','Comportamiento'], aggfunc={'Comportamiento':len})
    
    lista1.append(tabla1)
    lista2.append(tabla2)